## Set Up

In [12]:
import pandas as pd
import calendar
import datetime

In [13]:
mta190504_raw = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190504.txt")
mta190504_raw.describe()

,ENTRIES,EXITS
count,2.068570e+05,2.068570e+05
mean,4.057530e+07,3.319799e+07
std,2.082752e+08,1.926699e+08
min,0.000000e+00,0.000000e+00
25%,3.495460e+05,1.413340e+05
50%,2.176408e+06,1.241604e+06
75%,6.775342e+06,4.590174e+06
max,2.129343e+09,2.124127e+09


## Data Cleaning and Variable Creation

In [14]:
#Clean column names
mta190504_raw.columns = mta190504_raw.columns.str.strip()
mta190504_raw.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [15]:
# Go from raw DF to new DF that will have rows dropped
mta190504 = mta190504_raw.copy()

# Create variables:
# "time_hour" that simplifies the hour for later grouping purposes
mta190504["time_hour"] = pd.to_numeric(mta190504["TIME"].str[0:2])
# formatted date variable "DDATE"
mta190504['DDATE']=[datetime.datetime.strptime(x, '%m/%d/%Y') for x in mta190504['DATE']]
# formatted time variable "DTIME"
mta190504['DTIME']=[format(datetime.datetime.strptime(x, '%H:%M:%S'),"%H:%M:%S") for x in mta190504['TIME']]
# formatted day of the week variable "DDAY"
mta190504['DDAY']=[calendar.day_name[datetime.datetime.weekday(x)] for x in mta190504['DDATE']]

In [16]:
# Sort before grouping
mta190504.sort_values(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION","DATE","TIME","DESC"])
# Create difference columns to calculate difference in entries and exits between the row and the row before (aka the time before)
mta190504["entries_diff"] = mta190504.groupby(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION"]).ENTRIES.diff()
mta190504["exits_diff"] = mta190504.groupby(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION"]).EXITS.diff()
mta190504["entries-exits"] = mta190504["entries_diff"] - mta190504["exits_diff"]
mta190504["entries+exits"] = mta190504["entries_diff"] + mta190504["exits_diff"]
mta190504.describe()

,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits,entries+exits
count,2.068570e+05,2.068570e+05,206857.000000,2.020060e+05,2.020060e+05,2.020060e+05,2.020060e+05
mean,4.057530e+07,3.319799e+07,10.517256,2.075907e+03,7.433828e+03,-5.357922e+03,9.509735e+03
std,2.082752e+08,1.926699e+08,6.806081,3.508686e+06,3.415174e+06,1.754742e+06,6.698467e+06
min,0.000000e+00,0.000000e+00,0.000000,-8.392517e+08,-3.843496e+08,-6.714099e+08,-1.007094e+09
25%,3.495460e+05,1.413340e+05,4.000000,9.000000e+00,8.000000e+00,-2.500000e+01,2.700000e+01
50%,2.176408e+06,1.241604e+06,10.000000,7.200000e+01,5.200000e+01,1.000000e+00,1.680000e+02
75%,6.775342e+06,4.590174e+06,16.000000,2.390000e+02,1.660000e+02,9.700000e+01,4.550000e+02
max,2.129343e+09,2.124127e+09,23.000000,1.126553e+09,1.078346e+09,4.820644e+07,2.204899e+09


In [17]:
#keep only rows with positive entries_diff, exits_diff, and ENTRIES
mta190504= mta190504[mta190504.entries_diff > 0]
mta190504 = mta190504[mta190504.exits_diff > 0]
mta190504 = mta190504[mta190504.ENTRIES > 0]
mta190504.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165746 entries, 1 to 206772
Data columns (total 19 columns):
C/A              165746 non-null object
UNIT             165746 non-null object
SCP              165746 non-null object
STATION          165746 non-null object
LINENAME         165746 non-null object
DIVISION         165746 non-null object
DATE             165746 non-null object
TIME             165746 non-null object
DESC             165746 non-null object
ENTRIES          165746 non-null int64
EXITS            165746 non-null int64
time_hour        165746 non-null int64
DDATE            165746 non-null datetime64[ns]
DTIME            165746 non-null object
DDAY             165746 non-null object
entries_diff     165746 non-null float64
exits_diff       165746 non-null float64
entries-exits    165746 non-null float64
entries+exits    165746 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(3), object(11)
memory usage: 25.3+ MB


In [18]:
# Checks
# No negative values in entries_diff or exits_diff
mta190504.describe()

,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits,entries+exits
count,1.657460e+05,1.657460e+05,165746.000000,1.657460e+05,1.657460e+05,1.657460e+05,1.657460e+05
mean,2.718628e+07,2.017851e+07,10.914640,1.067828e+04,1.291025e+04,-2.231969e+03,2.358854e+04
std,1.592469e+08,1.345904e+08,6.713693,3.142692e+06,3.626145e+06,9.938631e+05,6.712907e+06
min,2.000000e+00,9.000000e+00,0.000000,1.000000e+00,1.000000e+00,-4.016999e+08,2.000000e+00
25%,6.770092e+05,3.798330e+05,5.000000,3.100000e+01,2.400000e+01,-4.000000e+01,8.200000e+01
50%,2.814794e+06,1.657638e+06,12.000000,1.160000e+02,7.800000e+01,1.200000e+01,2.460000e+02
75%,7.036559e+06,5.019826e+06,17.000000,2.900000e+02,2.020000e+02,1.340000e+02,5.330000e+02
max,2.115816e+09,2.037805e+09,23.000000,1.126553e+09,1.078346e+09,4.820644e+07,2.204899e+09


In [27]:
#create intervals: 01-04;05-08;09-12;13-16;17-20;21-00
#start with 01 instead of 00, assuming that most audits are on the hour so first element is inclusive
#and last element is exclusive
def time_interval(x):
    if x in [1,2,3,4]:
        return "01:00-04:59"
    elif x in [5,6,7,8]:
        return "05:00-08:59"
    elif x in [9,10,11,12]:
        return "09:00-12:59"
    elif x in [13,14,15,16]:
        return "13:00-16:59"
    elif x in [17,18,19,20]:
        return "17:00-20:59"
    elif x in [21,22,23,0]:
        return "21:00-00:59"

In [28]:
# create time period category "time_cat"
mta190504["time_cat"] = mta190504["time_hour"].apply(time_interval)
mta190504.head(50)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour,DDATE,DTIME,DDAY,entries_diff,exits_diff,entries-exits,entries+exits,time_cat
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840,4,2019-04-27,04:00:00,Saturday,20.0,7.0,13.0,27.0,01:00-04:59
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875,8,2019-04-27,08:00:00,Saturday,23.0,35.0,-12.0,58.0,05:00-08:59
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951,12,2019-04-27,12:00:00,Saturday,100.0,76.0,24.0,176.0,09:00-12:59
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020,16,2019-04-27,16:00:00,Saturday,259.0,69.0,190.0,328.0,13:00-16:59
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,20:00:00,REGULAR,7035930,2385070,20,2019-04-27,20:00:00,Saturday,279.0,50.0,229.0,329.0,17:00-20:59
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,00:00:00,REGULAR,7036100,2385087,0,2019-04-28,00:00:00,Sunday,170.0,17.0,153.0,187.0,21:00-00:59
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,04:00:00,REGULAR,7036119,2385088,4,2019-04-28,04:00:00,Sunday,19.0,1.0,18.0,20.0,01:00-04:59
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,08:00:00,REGULAR,7036125,2385103,8,2019-04-28,08:00:00,Sunday,6.0,15.0,-9.0,21.0,05:00-08:59
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,12:00:00,REGULAR,7036197,2385155,12,2019-04-28,12:00:00,Sunday,72.0,52.0,20.0,124.0,09:00-12:59
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,16:00:00,REGULAR,7036372,2385198,16,2019-04-28,16:00:00,Sunday,175.0,43.0,132.0,218.0,13:00-16:59


In [29]:
# For outlier entries and exit differences, replace with imputed mean for the remaining valid entries
# INSERT SHREYAK'S CODE

## Output grouped data

In [30]:
# Group by station
mta_station = mta190504.groupby(["STATION"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station

,entries_diff,exits_diff,entries+exits
STATION,,,
ST. GEORGE,1.126554e+09,1.078346e+09,2.204900e+09
BURNSIDE AV,6.065936e+08,1.008265e+09,1.614859e+09
TWENTY THIRD ST,1.854775e+06,2.614257e+07,2.799735e+07
34 ST-PENN STA,1.023199e+06,9.224320e+05,1.945631e+06
GRD CNTRL-42 ST,8.811170e+05,7.697670e+05,1.650884e+06
34 ST-HERALD SQ,7.033650e+05,6.613760e+05,1.364741e+06
14 ST-UNION SQ,6.509990e+05,6.133660e+05,1.264365e+06
TIMES SQ-42 ST,6.144630e+05,5.791380e+05,1.193601e+06
23 ST,6.662450e+05,4.653150e+05,1.131560e+06


In [31]:
# Group by station and date
mta_station_date = mta190504.groupby(["STATION","DDATE"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_date

entries_diff    exits_diff  entries+exits
STATION         DDATE                                                
ST. GEORGE      2019-05-02  1.126553e+09  1.078346e+09   2.204899e+09
BURNSIDE AV     2019-04-30  6.065412e+08  1.008236e+09   1.614777e+09
TWENTY THIRD ST 2019-04-28  5.708060e+05  8.238772e+06   8.809578e+06
                2019-04-30  4.842370e+05  6.882493e+06   7.366730e+06
                2019-04-29  3.880840e+05  5.502804e+06   5.890888e+06
                2019-04-27  3.819410e+05  5.491754e+06   5.873695e+06
34 ST-PENN STA  2019-05-02  1.756570e+05  1.570050e+05   3.326620e+05
                2019-05-01  1.755410e+05  1.566320e+05   3.321730e+05
                2019-04-30  1.726120e+05  1.507380e+05   3.233500e+05
                2019-05-03  1.676200e+05  1.550120e+05   3.226320e+05
                2019-04-29  1.663790e+05  1.440310e+05   3.104100e+05
GRD CNTRL-42 ST 2019-05-01  1.617530e+05  1.377850e+05   2.995380e+05
                2019-05-02  1.605290e+05  1.382240e+05   2.987530e+05
                2019-04-30  1.595070e+05  1.364900e+05   2.959970e+05
                2019-05-03  1.549080e+05  1.318880e+05   2.867960e+05
                2019-04-29  1.474820e+05  1.291250e+05   2.766070e+05
34 ST-HERALD SQ 2019-05-03  1.212470e+05  1.134130e+05   2.346600e+05
                2019-05-02  1.199410e+05  1.136780e+05   2.336190e+05
                2019-05-01  1.198350e+05  1.119590e+05   2.317940e+05
                2019-04-30  1.173190e+05  1.080880e+05   2.254070e+05
                2019-04-29  1.089350e+05  1.025440e+05   2.114790e+05
14 ST-UNION SQ  2019-05-01  1.098340e+05  1.011380e+05   2.109720e+05
                2019-05-03  1.091300e+05  1.010290e+05   2.101590e+05
                2019-05-02  1.094200e+05  1.002190e+05   2.096390e+05
                2019-04-30  1.053080e+05  9.903900e+04   2.043470e+05
TIMES SQ-42 ST  2019-05-01  1.066730e+05  9.602200e+04   2.026950e+05
                2019-05-02  1.046070e+05  9.631300e+04   2.009200e+05
                2019-05-03  1.042200e+05  9.610300e+04   2.003230e+05
FULTON ST       2019-05-02  1.075740e+05  8.967000e+04   1.972440e+05
                2019-05-01  1.069580e+05  8.931700e+04   1.962750e+05
...                                  ...           ...            ...
BEACH 105 ST    2019-05-01  2.360000e+02  2.590000e+02   4.950000e+02
                2019-04-30  2.320000e+02  2.520000e+02   4.840000e+02
BROAD CHANNEL   2019-05-02  3.780000e+02  7.200000e+01   4.500000e+02
GRANT AV        2019-04-27  9.300000e+01  3.550000e+02   4.480000e+02
BROAD CHANNEL   2019-05-03  3.400000e+02  8.500000e+01   4.250000e+02
                2019-04-30  3.350000e+02  6.900000e+01   4.040000e+02
                2019-05-01  3.090000e+02  8.200000e+01   3.910000e+02
NEWARK C        2019-04-28  3.410000e+02  4.700000e+01   3.880000e+02
BROAD CHANNEL   2019-04-29  2.890000e+02  7.900000e+01   3.680000e+02
OZONE PK LEFFRT 2019-04-27  1.060000e+02  2.600000e+02   3.660000e+02
BEACH 98 ST     2019-04-27  1.570000e+02  1.330000e+02   2.900000e+02
PATH WTC 2      2019-04-27  2.020000e+02  3.800000e+01   2.400000e+02
80 ST           2019-04-27  2.400000e+01  1.920000e+02   2.160000e+02
PATH WTC 2      2019-04-28  1.820000e+02  2.800000e+01   2.100000e+02
BEACH 98 ST     2019-04-28  9.800000e+01  9.100000e+01   1.890000e+02
BROAD CHANNEL   2019-04-27  7.000000e+01  7.400000e+01   1.440000e+02
AQUEDUCT RACETR 2019-04-27  1.330000e+02  1.000000e+01   1.430000e+02
BROAD CHANNEL   2019-04-28  6.500000e+01  6.300000e+01   1.280000e+02
BEACH 105 ST    2019-04-27  5.300000e+01  7.400000e+01   1.270000e+02
ROCKAWAY BLVD   2019-04-27  4.200000e+01  7.800000e+01   1.200000e+02
157 ST          2019-04-27  5.200000e+01  6.400000e+01   1.160000e+02
AQUEDUCT RACETR 2019-04-28  5.200000e+01  3.500000e+01   8.700000e+01
88 ST           2019-04-27  2.500000e+01  5.900000e+01   8.400000e+01
BEACH 105 ST    2019-04-28  3.600000e+01  4.600000e+01   8.200000e+01
AQUEDUCT N.COND 2019-04-27  1.600000e+

In [32]:
# Group by station and day of week
mta_station_day = mta190504.groupby(["STATION","DDAY"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_day

entries_diff    exits_diff  entries+exits
STATION         DDAY                                                
ST. GEORGE      Thursday   1.126553e+09  1.078346e+09   2.204899e+09
BURNSIDE AV     Tuesday    6.065412e+08  1.008236e+09   1.614777e+09
TWENTY THIRD ST Sunday     5.708060e+05  8.238772e+06   8.809578e+06
                Tuesday    4.842370e+05  6.882493e+06   7.366730e+06
                Monday     3.880840e+05  5.502804e+06   5.890888e+06
                Saturday   3.819410e+05  5.491754e+06   5.873695e+06
34 ST-PENN STA  Thursday   1.756570e+05  1.570050e+05   3.326620e+05
                Wednesday  1.755410e+05  1.566320e+05   3.321730e+05
                Tuesday    1.726120e+05  1.507380e+05   3.233500e+05
                Friday     1.676200e+05  1.550120e+05   3.226320e+05
                Monday     1.663790e+05  1.440310e+05   3.104100e+05
GRD CNTRL-42 ST Wednesday  1.617530e+05  1.377850e+05   2.995380e+05
                Thursday   1.605290e+05  1.382240e+05   2.987530e+05
                Tuesday    1.595070e+05  1.364900e+05   2.959970e+05
                Friday     1.549080e+05  1.318880e+05   2.867960e+05
                Monday     1.474820e+05  1.291250e+05   2.766070e+05
34 ST-HERALD SQ Friday     1.212470e+05  1.134130e+05   2.346600e+05
                Thursday   1.199410e+05  1.136780e+05   2.336190e+05
                Wednesday  1.198350e+05  1.119590e+05   2.317940e+05
                Tuesday    1.173190e+05  1.080880e+05   2.254070e+05
                Monday     1.089350e+05  1.025440e+05   2.114790e+05
14 ST-UNION SQ  Wednesday  1.098340e+05  1.011380e+05   2.109720e+05
                Friday     1.091300e+05  1.010290e+05   2.101590e+05
                Thursday   1.094200e+05  1.002190e+05   2.096390e+05
                Tuesday    1.053080e+05  9.903900e+04   2.043470e+05
TIMES SQ-42 ST  Wednesday  1.066730e+05  9.602200e+04   2.026950e+05
                Thursday   1.046070e+05  9.631300e+04   2.009200e+05
                Friday     1.042200e+05  9.610300e+04   2.003230e+05
FULTON ST       Thursday   1.075740e+05  8.967000e+04   1.972440e+05
                Wednesday  1.069580e+05  8.931700e+04   1.962750e+05
...                                 ...           ...            ...
BEACH 105 ST    Wednesday  2.360000e+02  2.590000e+02   4.950000e+02
                Tuesday    2.320000e+02  2.520000e+02   4.840000e+02
BROAD CHANNEL   Thursday   3.780000e+02  7.200000e+01   4.500000e+02
GRANT AV        Saturday   9.300000e+01  3.550000e+02   4.480000e+02
BROAD CHANNEL   Friday     3.400000e+02  8.500000e+01   4.250000e+02
                Tuesday    3.350000e+02  6.900000e+01   4.040000e+02
                Wednesday  3.090000e+02  8.200000e+01   3.910000e+02
NEWARK C        Sunday     3.410000e+02  4.700000e+01   3.880000e+02
BROAD CHANNEL   Monday     2.890000e+02  7.900000e+01   3.680000e+02
OZONE PK LEFFRT Saturday   1.060000e+02  2.600000e+02   3.660000e+02
BEACH 98 ST     Saturday   1.570000e+02  1.330000e+02   2.900000e+02
PATH WTC 2      Saturday   2.020000e+02  3.800000e+01   2.400000e+02
80 ST           Saturday   2.400000e+01  1.920000e+02   2.160000e+02
PATH WTC 2      Sunday     1.820000e+02  2.800000e+01   2.100000e+02
BEACH 98 ST     Sunday     9.800000e+01  9.100000e+01   1.890000e+02
BROAD CHANNEL   Saturday   7.000000e+01  7.400000e+01   1.440000e+02
AQUEDUCT RACETR Saturday   1.330000e+02  1.000000e+01   1.430000e+02
BROAD CHANNEL   Sunday     6.500000e+01  6.300000e+01   1.280000e+02
BEACH 105 ST    Saturday   5.300000e+01  7.400000e+01   1.270000e+02
ROCKAWAY BLVD   Saturday   4.200000e+01  7.800000e+01   1.200000e+02
157 ST          Saturday   5.200000e+01  6.400000e+01   1.160000e+02
AQUEDUCT RACETR Sunday     5.200000e+01  3.500000e+01   8.700000e+01
88 ST           Saturday   2.500000e+01  5.900000e+01   8.400000e+01
BEACH 105 ST    Sunday     3.600000e+01  4.600000e+01   8.200000e+01
AQUEDUCT N.COND Saturday   1.600000e+01  5.700000e+01   7.300000e+01
ST. GEORGE      Sunday   

In [33]:
# Group by station, day of week, and time of day
mta_station_day_time = mta190504.groupby(["STATION","DDAY","time_cat"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_day_time

entries_diff    exits_diff  \
STATION         DDAY      time_cat                                  
ST. GEORGE      Thursday  13:00-16:59  1.126553e+09  1.078346e+09   
BURNSIDE AV     Tuesday   09:00-12:59  6.065334e+08  1.008232e+09   
TWENTY THIRD ST Tuesday   17:00-20:59  9.862100e+04  1.377162e+06   
                Monday    17:00-20:59  9.895200e+04  1.375547e+06   
                Tuesday   09:00-12:59  9.571400e+04  1.377954e+06   
                Monday    09:00-12:59  9.546200e+04  1.377769e+06   
                Tuesday   13:00-16:59  9.549400e+04  1.376928e+06   
                Monday    13:00-16:59  9.576800e+04  1.374993e+06   
                Tuesday   05:00-08:59  9.499700e+04  1.375028e+06   
                          01:00-04:59  9.518600e+04  1.374559e+06   
                Saturday  17:00-20:59  9.605200e+04  1.373243e+06   
                Monday    05:00-08:59  9.488700e+04  1.374314e+06   
                Sunday    17:00-20:59  9.562300e+04  1.373512e+06   
                          21:00-00:59  9.558400e+04  1.373260e+06   
                Saturday  21:00-00:59  9.564500e+04  1.373180e+06   
                Sunday    13:00-16:59  9.508200e+04  1.373609e+06   
                Saturday  13:00-16:59  9.530900e+04  1.372756e+06   
                Sunday    01:00-04:59  9.513900e+04  1.372792e+06   
                          09:00-12:59  9.471600e+04  1.372987e+06   
                Saturday  09:00-12:59  9.480900e+04  1.372506e+06   
                Sunday    05:00-08:59  9.466200e+04  1.372612e+06   
34 ST-PENN STA  Thursday  17:00-20:59  4.573600e+04  6.133400e+04   
                Wednesday 17:00-20:59  4.486000e+04  6.214900e+04   
                Friday    17:00-20:59  4.508000e+04  5.930500e+04   
                Tuesday   17:00-20:59  4.401800e+04  5.935700e+04   
                Monday    17:00-20:59  4.326100e+04  5.925400e+04   
                Tuesday   09:00-12:59  6.071000e+04  3.322300e+04   
                Wednesday 09:00-12:59  5.939900e+04  3.362900e+04   
                Thursday  09:00-12:59  5.881100e+04  3.382900e+04   
34 ST-HERALD SQ Thursday  17:00-20:59  5.088700e+04  3.316700e+04   
...                                             ...           ...   
BEACH 105 ST    Saturday  05:00-08:59  4.000000e+00  6.000000e+00   
NEPTUNE AV      Saturday  05:00-08:59  7.000000e+00  2.000000e+00   
BROAD CHANNEL   Monday    05:00-08:59  6.000000e+00  3.000000e+00   
ST. GEORGE      Monday    21:00-00:59  6.000000e+00  3.000000e+00   
BROAD CHANNEL   Monday    01:00-04:59  5.000000e+00  4.000000e+00   
NEWARK HW BMEBE Sunday    01:00-04:59  5.000000e+00  4.000000e+00   
LACKAWANNA      Sunday    05:00-08:59  2.000000e+00  7.000000e+00   
PATH WTC 2      Saturday  05:00-08:59  1.000000e+00  8.000000e+00   
RIT-MANHATTAN   Friday    05:00-08:59  7.000000e+00  1.000000e+00   
RIT-ROOSEVELT   Saturday  05:00-08:59  5.000000e+00  3.000000e+00   
AQUEDUCT RACETR Sunday    05:00-08:59  4.000000e+00  4.000000e+00   
190 ST          Friday    01:00-04:59  5.000000e+00  2.000000e+00   
HUNTS POINT AV  Monday    09:00-12:59  5.000000e+00  1.000000e+00   
AQUEDUCT N.COND Saturday  21:00-00:59  3.000000e+00  3.000000e+00   
WILSON AV       Thursday  21:00-00:59  1.000000e+00  5.000000e+00   
80 ST           Saturday  09:00-12:59  3.000000e+00  2.000000e+00   
BROAD CHANNEL   Sunday    01:00-04:59  3.000000e+00  2.000000e+00   
BEACH 105 ST    Sunday    05:00-08:59  2.000000e+00  3.000000e+00   
BROAD CHANNEL   Friday    13:00-16:59  2.000000e+00  3.000000e+00   
BEACH 105 ST    Sunday    01:00-04:59  3.000000e+00  1.000000e+00   
AQUEDUCT N.COND Sunday    01:00-04:59  2.000000e+00  1.000000e+00   
PATH WTC 2      Sunday    05:00-08:59  1.000000e+00  2.000000e+00   
RIT-MANHATTAN   Tuesday   05:00-08:59  1.000000e+00  2.000000e+00   
157 ST          Saturday  17:00-20:59  1.000000e+00  1.000000e+00   
80 ST           Saturday  17:00-20:59  1.000000e+00  1.000000e+00   
AQUEDUCT RACETR Saturday  05:00-08:59